In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
#from langchain_core.messages import Humanmeesages
prompt_template = ChatPromptTemplate([
    ('system', """You are a professional chatbot working at a horse racing track, providing detailed, helpful, and dynamic information based on the following sources:
    Quick Information Access: Summarize key details such as race schedules, race types, and betting methods in an easy-to-understand way for beginners.
    "Please provide the information based on the following websites:

https://namu.wiki/w/%EA%B2%BD%EB%A7%88/%ED%95%9C%EA%B5%AD
https://race.kra.co.kr/raceguide/RaceProcessGetService.do
https://knetz.kra.co.kr/index.do
https://park.kra.co.kr/seoul_main.do?strToken=XXXXX
http://www.okrace.co.kr/okrace/ai_info.php"
    Dynamic Betting Guidance: Provide tips for different types of bets (Win, Place, Exacta, etc.) and display real-time odds.
    Location-Specific Information: Offer details about the specific racetrack, including facilities, events, and location-based info.
    Track-Specific Information: Customize responses based on unique features and events of each racetrack.
    Ensure that all responses are friendly, accurate, and cater to both newcomers and regular visitors."""),
    
    ('user', '{question}')
])

# 출력 파서 생성
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

## pdf 파일 로드

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_community.document_loaders import PyPDFLoader
import os

# 여러 PDF 파일 경로를 리스트로 생성
pdf_folder = 'C:/Workspaces/project_ma/pdfs'
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

# 각 PDF 파일을 로드하여 문서로 처리
pdf_documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents.extend(loader.load())  # 문서를 리스트에 추가

print(f"Loaded {len(pdf_documents)} documents.")

Loaded 203 documents.


## XML 파일 로드

In [34]:
from langchain_community.document_loaders import UnstructuredXMLLoader
import os
# 여러 xml 파일 경로를 리스트로 생성
xml_folder = 'C:/Workspaces/project_ma/xmls'
xml_files = [os.path.join(xml_folder, file) for file in os.listdir(xml_folder) if file.endswith('.xml')]

# xml 파일들 로드
xml_documents = []
for xml_file in xml_files:
    loader = UnstructuredXMLLoader(xml_file)
    xml_documents.extend(loader.load())  # 문서를 리스트에 추가

print(f"Loaded {len(xml_documents)} documents.")

Loaded 1 documents.


In [47]:
type(xml_documents)

langchain_core.documents.base.Document

In [48]:
xml_documents[0].replace('\n', ' ')


AttributeError: 'list' object has no attribute 'replace'

## 데이터 병합 + 임베딩 모델 호출

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

all_documents = pdf_documents + xml_documents

## Vector Store 구축 - FAISS 사용

In [36]:
from langchain.vectorstores import FAISS

vectors = FAISS.from_documents(all_documents, embeddings) #documents : Document
vectors

## Retriever 객체 생성

In [37]:
retriever = vectors.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012A47C545C0>, search_kwargs={})

In [38]:
# 벡터스토어 검색 및 LLM 질의

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

#질의 객체
rtvr_qa=RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type = 'stuff' #검색된 내용을 하나의 텍스트로 생성해서 LLM에 전달
)

#llm 질의
response1 = rtvr_qa.invoke('2024년 12월 20일 부경 경마장 경기에 대해 알려줘')
print(response1)

{'query': '2024년 12월 20일 부경 경마장 경기에 대해 알려줘', 'result': '2024년 12월 20일은 경마 시행 계획에 따르면 부경 경마장에서 경기가 열리지 않는 날입니다. 12월 20일은 금요일로, 부경 경마장은 금요일에 경기를 시행하지 않습니다.'}


In [28]:
#response1 = rtvr_qa.invoke('오토벳에 대해 알려줘')
#print(response1)
#TODO pdf 업로드 시, 내용이 누락될 가능성?
#race_guide.pdf에 오토벳에 대한 내용이 있는데 뱉어주는 게 없다.
#민하님께서 guide.pdf만 넣고 돌렸을 땐 관련 내용이 잘 나왔는데, 여러 pdf 파일을 넣고 돌리니깐 해당 내용을 제공할 수 없단 답변을 받았다고 하심.
#이에 다 같이 돌려봤으나 똑같은 증상을 겪었다.